## Análise de Negócios
### Preços dos combustíveis ofertados nos postos

<img src = 'imagens/preco_combustivel.jpeg' width=400>

&nbsp;

### Fomos contratados por uma Petroleira para construir uma análise de negócios para entender a dinâmica de preços que está sendo ofertado pelos postos.

1 - Teremos que trabalhar com Dados Existentes por meio deste link: <br/>
https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis


<img src = 'imagens/anp.png' width=600>

&nbsp;

Como foi uma solicitação do nosso cliente, teremos que avaliar os ultimos 5 anos da base de dados.<br/>

<img src = 'imagens/tabela_base_dados.png' height=400>

&nbsp;

Quando abrimos as tabelas nos deparamos que cada semestre possui uma quantidade muito grande de dados, logo juntas 5 anos (10 tabelas) fica difícil!

Como muitas das vezes não temos o Engenheiro de Dados, temos que dar os nossos pulos!<br/>


2 - Armazenar esses dados em um banco de dados

&nbsp;

<img src = 'imagens/PostgreSQL-logo.png' height=250> <img src = 'imagens/postgre1.png' height=250>

In [ ]:
"""
CREATE TABLE anp.preco_combustivel(
	regiao 			varchar(255),
	estado			varchar(255),
	municipio		varchar(255),
	revenda			varchar(255),
	cnpj			varchar(255),
	nome_rua		varchar(255),
	numero_rua		varchar(255),
	complemento		varchar(255),
	bairro			varchar(255),
	cep				varchar(255),
	produto			varchar(255),
	data_coleta		date,
	valor_venda		float,
	unidade_medida	varchar(255),
	bandeira		varchar(255)
)
"""

### Processo de ETL dos dados no Banco

Para isso utilizaremos o Knime

&nbsp;

<img src = 'imagens/knime.png' height=250> <img src = 'imagens/knime2.png' height=250><br/>

&nbsp;

Para atender as nossas necessidades vamos construir desta forma.


<img src = 'imagens/knime3.png' width=400><br/>

3 - Utilizar esses dados para as nossas análises

### Análise dos dados

#### Primeiros Passos

In [ ]:
# pip install pandas
# pip install psycopg2

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = ps.connect(dbname = 'ANP',
                    user ='postgres',
                    password = 'postgres',
                    host = 'localhost',
                    port = '5432' )

In [3]:
sql = """
SELECT * FROM anp.preco_combustivel
"""

In [4]:
df = sqlio.read_sql_query(sql, conn)

In [5]:
df.head()

,regiao,estado,municipio,revenda,cnpj,nome_rua,numero_rua,complemento,bairro,cep,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,GASOLINA,2019-01-03,4.199,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
1,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,ETANOL,2019-01-03,2.899,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
2,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,DIESEL S10,2019-01-03,3.349,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
3,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,GNV,2019-01-03,2.439,R$ / m³,PETROBRAS DISTRIBUIDORA S.A.
4,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,None,CENTRO,92310-000,GASOLINA,2019-01-02,4.399,R$ / litro,BRANCA


In [9]:
df["data_coleta"] = pd.to_datetime(df["data_coleta"])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4342368 entries, 0 to 4342367
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   regiao          object        
 1   estado          object        
 2   municipio       object        
 3   revenda         object        
 4   cnpj            object        
 5   nome_rua        object        
 6   numero_rua      object        
 7   complemento     object        
 8   bairro          object        
 9   cep             object        
 10  produto         object        
 11  data_coleta     datetime64[ns]
 12  valor_venda     float64       
 13  unidade_medida  object        
 14  bandeira        object        
dtypes: datetime64[ns](1), float64(1), object(13)
memory usage: 496.9+ MB


In [12]:
df.isnull().sum()

regiao                  0
estado                  0
municipio               0
revenda                 0
cnpj                    0
nome_rua                0
numero_rua           2201
complemento       3336352
bairro              12140
cep                     0
produto                 0
data_coleta             0
valor_venda             0
unidade_medida          0
bandeira                0
dtype: int64

Os dados mais importantes estão completos!

In [13]:
df_anp = df[["data_coleta", "regiao", "estado", "municipio", "bandeira", "produto", "valor_venda"]]

In [19]:
df_anp

,data_coleta,regiao,estado,municipio,bandeira,produto,valor_venda,ano,mes
0,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA,4.199,2019,1
1,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,ETANOL,2.899,2019,1
2,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10,3.349,2019,1
3,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,GNV,2.439,2019,1
4,2019-01-02,S,RS,CANOAS,BRANCA,GASOLINA,4.399,2019,1
...,...,...,...,...,...,...,...,...,...
4342363,2023-12-28,N,AP,SANTANA,IPIRANGA,DIESEL,6.440,2023,12
4342364,2023-12-28,N,AP,SANTANA,IPIRANGA,DIESEL S10,6.490,2023,12
4342365,2023-12-27,NE,PI,PARNAIBA,BRANCA,GASOLINA,5.200,2023,12
4342366,2023-12-27,NE,PI,PARNAIBA,BRANCA,GASOLINA ADITIVADA,5.200,2023,12


In [15]:
df_anp["ano"] = df_anp["data_coleta"].dt.year
df_anp["mes"] = df_anp["data_coleta"].dt.month

In [16]:
df_anp.head()

,data_coleta,regiao,estado,municipio,bandeira,produto,valor_venda,ano,mes
0,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA,4.199,2019,1
1,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,ETANOL,2.899,2019,1
2,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10,3.349,2019,1
3,2019-01-03,SE,SP,GUARULHOS,PETROBRAS DISTRIBUIDORA S.A.,GNV,2.439,2019,1
4,2019-01-02,S,RS,CANOAS,BRANCA,GASOLINA,4.399,2019,1


#### Investigando os dados

In [20]:
# Estatística Descritiva dos valores numéricos
df_anp[["valor_venda","ano","mes"]].describe().round(2)

,valor_venda,ano,mes
count,4342368.00,4342368.00,4342368.00
mean,4.83,2021.00,6.30
std,1.29,1.46,3.36
min,1.80,2019.00,1.00
25%,3.77,2020.00,3.00
50%,4.69,2021.00,6.00
75%,5.79,2022.00,9.00
max,9.79,2023.00,12.00


#### Quais tipos de combustíveis são comercializados?

In [22]:
print(df_anp.produto.unique())

['GASOLINA' 'ETANOL' 'DIESEL S10' 'GNV' 'DIESEL' 'GASOLINA ADITIVADA']


In [25]:
# Quantidade de registros por combustível
df_anp.produto.value_counts()

produto
GASOLINA              1184447
ETANOL                1043954
DIESEL S10             945165
DIESEL                 567531
GASOLINA ADITIVADA     521467
GNV                     79804
Name: count, dtype: int64

#### Investigando os valores mínimos, máximos e médios dos produtos por ano

In [26]:
df_anp_valor = df_anp[["ano", "produto", "valor_venda"]]

In [27]:
df_anp_valor.groupby(["produto", "ano"]).agg(["min", "max", "mean"]).round(2)

valor_venda            
                                min   max  mean
produto            ano                         
DIESEL             2019        2.87  4.99  3.60
                   2020        2.45  5.09  3.42
                   2021        3.10  6.99  4.69
                   2022        3.14  9.00  6.63
                   2023        3.97  7.99  5.76
DIESEL S10         2019        2.79  5.09  3.69
                   2020        2.46  5.38  3.51
                   2021        2.80  6.96  4.74
                   2022        3.59  9.65  6.73
                   2023        4.19  9.00  5.86
ETANOL             2019        2.10  5.47  3.17
                   2020        1.80  5.15  3.18
                   2021        2.05  7.90  4.67
                   2022        2.49  7.98  4.85
                   2023        2.69  6.96  4.00
GASOLINA           2019        3.39  6.29  4.42
                   2020        2.87  5.90  4.28
                   2021        3.10  8.00  5.89
                   2022        3.49  8.99  6.32
                   2023        4.09  8.19  5.53
GASOLINA ADITIVADA 2020        3.69  6.00  4.59
                   2021        3.46  8.99  6.04
                   2022        4.09  9.28  6.47
                   2023        3.47  9.79  5.71
GNV                2019        2.00  4.56  3.22
                   2020        2.00  4.78  3.17
                   2021        2.38  6.70  3.89
                   2022        3.17  7.99  5.04
                   2023        2.39  6.71  4.64

In [34]:
df_anp_valor_regiao = df_anp[["ano", "produto", "regiao", "valor_venda"]]

In [35]:
df_anp_valor_regiao.head()

,ano,produto,regiao,valor_venda
0,2019,GASOLINA,SE,4.199
1,2019,ETANOL,SE,2.899
2,2019,DIESEL S10,SE,3.349
3,2019,GNV,SE,2.439
4,2019,GASOLINA,S,4.399


In [36]:
df_anp_valor_regiao.groupby(["produto", "ano","regiao"]).agg(["min", "max", "mean"]).round(2)

valor_venda            
                            min   max  mean
produto ano  regiao                        
DIESEL  2019 CO            3.02  4.54  3.73
             N             3.10  4.99  3.92
             NE            2.99  4.65  3.67
             S             2.89  4.49  3.45
             SE            2.87  4.99  3.54
...                         ...   ...   ...
GNV     2023 CO            3.19  6.69  4.19
             N             3.59  5.50  4.36
             NE            2.39  6.28  4.34
             S             3.24  6.59  5.18
             SE            3.49  6.71  4.63

[145 rows x 3 columns]

#### Quantidade de bandeiras por produto/ano

In [37]:
df_anp_bandeira = df_anp[["ano", "bandeira", "produto"]]
df_anp_bandeira.head()

,ano,bandeira,produto
0,2019,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA
1,2019,PETROBRAS DISTRIBUIDORA S.A.,ETANOL
2,2019,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10
3,2019,PETROBRAS DISTRIBUIDORA S.A.,GNV
4,2019,BRANCA,GASOLINA


In [43]:
df_anp_bandeira.groupby(["ano", "produto"])[["bandeira"]].nunique()

bandeira
ano  produto                     
2019 DIESEL                    59
     DIESEL S10                61
     ETANOL                    63
     GASOLINA                  64
     GNV                       22
2020 DIESEL                    52
     DIESEL S10                57
     ETANOL                    58
     GASOLINA                  58
     GASOLINA ADITIVADA        43
     GNV                       23
2021 DIESEL                    54
     DIESEL S10                54
     ETANOL                    55
     GASOLINA                  55
     GASOLINA ADITIVADA        54
     GNV                       22
2022 DIESEL                    51
     DIESEL S10                60
     ETANOL                    60
     GASOLINA                  60
     GASOLINA ADITIVADA        58
     GNV                       24
2023 DIESEL                    43
     DIESEL S10                46
     ETANOL                    47
     GASOLINA                  48
     GASOLINA ADITIVADA        47
     GNV                       22

In [44]:
df_anp_bandeira_regiao = df_anp[["ano", "bandeira", "produto", "regiao"]]
df_anp_bandeira_regiao.head()

,ano,bandeira,produto,regiao
0,2019,PETROBRAS DISTRIBUIDORA S.A.,GASOLINA,SE
1,2019,PETROBRAS DISTRIBUIDORA S.A.,ETANOL,SE
2,2019,PETROBRAS DISTRIBUIDORA S.A.,DIESEL S10,SE
3,2019,PETROBRAS DISTRIBUIDORA S.A.,GNV,SE
4,2019,BRANCA,GASOLINA,S


#### Quantidade de bandeiras de DIESEL por ano nas regiões

In [54]:
df_anp_bandeira_regiao[df_anp_bandeira_regiao["produto"] == "DIESEL"].groupby(["ano", "regiao"])[["bandeira"]].nunique()

bandeira
ano  regiao          
2019 CO            18
     N             11
     NE            17
     S             23
     SE            21
2020 CO            15
     N             10
     NE            17
     S             20
     SE            18
2021 CO            21
     N             11
     NE            19
     S             21
     SE            19
2022 CO            21
     N             11
     NE            17
     S             21
     SE            16
2023 CO            14
     N             11
     NE            18
     S             20
     SE            13